## Modifiziere die Klasse <code>player</code> so, dass sie
- ein Attribut drink_hist hat, das mit [ ] initalisiert wird
- am Ende eines jeden Zeitschritts diese Liste mit .append(...) einen neuen Eintrag bekommt, der den bisher getrunkenen Getränken entspricht

In [ ]:
import numpy as np

In [ ]:
class player():
    def __init__(self, name, precision, speed, stat_msg):
        self.name = name
        self.precision = precision
        self.speed = speed
        self.stat_msg = stat_msg
        self.tries = 0
        self.num_balls = 0
        self.ndrinks = 0
        self.wait = 0
    def say_name(self):
        print(f"Ich heiße {self.name}!")
    def print_info(self):
        print(f"Ich heiße {self.name}, habe die Präzision {self.precision:.2f} "
              + f"und die Geschwindigkeit {self.speed:.2f}.")
    def getting_ball(self):
        self.num_balls += 1
        self.wait = 1
    def drink(self):
        if self.stat_msg: print(f"{self.name} drinking...")
        self.ndrinks += 1
        self.wait = 2
    def throw(self):
        if self.stat_msg: print(f"{self.name} throwing... ", end="")
        self.tries += 1
        success = (np.random.random() < self.precision)
        if self.stat_msg:
            if success:
                print("success!")
            else:
                print("no success!")
        return success
    def time_step(self):
        if self.num_balls == 1:
            if not self.wait:
                success = self.throw()
                if success:
                    if self.left_neighbour.num_balls > 0:
                        # left neighbour has a ball, catch him/her!
                        self.left_neighbour.getting_ball()
                        self.num_balls -= 1
                        if self.stat_msg: print(f"{self.name} caught {self.left_neighbour.name}!")
                    else:
                        # just move to the left neighbour
                        self.left_neighbour.getting_ball()
                        self.num_balls -= 1
                        if self.stat_msg: print(f"moving from {self.name} to {self.left_neighbour.name}!")
        elif self.num_balls == 2:
            # drink, keep one ball, pass on the other
            self.drink()
            self.left_neighbour.getting_ball()
            self.num_balls -= 1
        if self.wait: self.wait -= 1

In [ ]:
class table():
    def __init__(self, num_players, min_prec=0.1, max_prec=1,
                 min_speed=0.2, max_speed=1, ndrinks = 20,
                 stat_msg=True):
        self.players = []
        self.ndrinks = ndrinks
        self.stat_msg = stat_msg
        # initialize the players
        for ii in range(num_players):
            self.players.append(player(f"p{ii}", 
                    precision=np.random.random() * (max_prec-min_prec) + min_prec ,
                    speed=np.random.random() * (max_speed-min_speed) + min_speed,
                    stat_msg=stat_msg))
        # arrange them around the table by defining the left neighbour of each player
        for ii, pl in enumerate(self.players[:-1]):
            pl.left_neighbour = self.players[ii+1]
        # and for the last one, to close the loop:
        self.players[-1].left_neighbour = self.players[0]
        # initialize the starting position of the two balls:
        index_1 = 0
        index_2 = int(len(self.players)/2)
        self.players[index_1].num_balls = 1
        self.players[index_2].num_balls = 1
    def print_info(self):
        for pl in self.players:
            pl.print_info()
    def time_step(self):
        for pl in self.players:
            pl.time_step()
    def run_game(self, nsteps=20):
        for ii in range(nsteps):
            if self.stat_msg: print(f"time step {ii}")
            if self.ndrinks > 0:
                self.time_step()
            else:
                break

In [ ]:
# Überprüfen der Funktionalitöt
tbl = table(num_players=5, stat_msg=False)
tbl.run_game()
tbl.players[0].drink_hist